# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [23]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

### Генерация датасета

In [6]:
iris = datasets.load_iris()
print(iris.DESCR)

Iris Plants Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)
    ============== ==== ==== ======= ===== ====================

    :Missing Attribute Values: None
  

In [7]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                    test_size = 0.3, random_state = 0)

### Задание модели

In [8]:
classifier = linear_model.SGDClassifier(random_state = 0)

### Генерация сетки

In [11]:
classifier.get_params().keys()

dict_keys(['tol', 'epsilon', 'class_weight', 'fit_intercept', 'max_iter', 'n_iter', 'learning_rate', 'random_state', 'loss', 'n_jobs', 'power_t', 'alpha', 'shuffle', 'penalty', 'verbose', 'l1_ratio', 'eta0', 'warm_start', 'average'])

In [26]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : list(range(5, 10)),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

In [19]:
cv = model_selection.StratifiedShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

In [37]:
grid_cv = model_selection.GridSearchCV(estimator=classifier,
                                       param_grid=parameters_grid,
                                       scoring = 'accuracy',
                                       cv = cv,
                                       return_train_score=True)

In [38]:
%%time
grid_cv.fit(train_data, train_labels)

CPU times: user 6.3 s, sys: 28.2 ms, total: 6.33 s
Wall time: 6.36 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
       error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=0, shuffle=True,
       tol=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_iter': [5, 6, 7, 8, 9], 'penalty': ['l1', 'l2'], 'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'], 'alpha': array([ 0.0001 ,  0.00032,  0.00055,  0.00078,  0.001  ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [39]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.00032499999999999999, average=False, class_weight=None,
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=9, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=0, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [40]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.895238095238
{'max_iter': 9, 'penalty': 'l1', 'loss': 'hinge', 'alpha': 0.00032499999999999999}


In [42]:
cv_results = pd.DataFrame(grid_cv.cv_results_)
cv_results.head()

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,param_loss,param_max_iter,param_penalty,params,rank_test_score,...,split7_test_score,split7_train_score,split8_test_score,split8_train_score,split9_test_score,split9_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.001417,0.000341,0.757143,0.728571,0.0001,hinge,5,l1,"{'max_iter': 5, 'penalty': 'l1', 'loss': 'hing...",57,...,0.714286,0.690476,0.619048,0.630952,0.619048,0.630952,0.000405,0.000119,0.135443,0.107644
1,0.001307,0.000305,0.666667,0.616667,0.0001,hinge,5,l2,"{'max_iter': 5, 'penalty': 'l2', 'loss': 'hing...",114,...,0.714286,0.690476,0.523810,0.416667,0.904762,0.928571,0.000343,0.000073,0.159364,0.177521
2,0.001444,0.000375,0.614286,0.582143,0.0001,hinge,6,l1,"{'max_iter': 6, 'penalty': 'l1', 'loss': 'hing...",139,...,0.476190,0.392857,0.333333,0.321429,0.428571,0.369048,0.000201,0.000090,0.143570,0.149114
3,0.001482,0.000335,0.685714,0.682143,0.0001,hinge,6,l2,"{'max_iter': 6, 'penalty': 'l2', 'loss': 'hing...",97,...,0.619048,0.607143,0.333333,0.333333,0.904762,0.928571,0.000270,0.000046,0.153862,0.159990
4,0.001414,0.000372,0.795238,0.784524,0.0001,hinge,7,l1,"{'max_iter': 7, 'penalty': 'l1', 'loss': 'hing...",35,...,0.952381,0.880952,0.952381,0.964286,0.714286,0.690476,0.000179,0.000132,0.165026,0.146912


#### Randomized grid search

In [43]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier,
                                                        parameters_grid,
                                                        scoring = 'accuracy',
                                                        cv = cv,
                                                        n_iter = 20, 
                                                        random_state = 0)

In [44]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

CPU times: user 619 ms, sys: 5.94 ms, total: 625 ms
Wall time: 628 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
          error_score='raise',
          estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=0, shuffle=True,
       tol=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=20, n_jobs=1,
          param_distributions={'max_iter': [5, 6, 7, 8, 9], 'penalty': ['l1', 'l2'], 'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'], 'alpha': array([ 0.0001 ,  0.00032,  0.00055,  0.00078,  0.001  ])},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [45]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.814285714286
{'max_iter': 9, 'penalty': 'l1', 'loss': 'log', 'alpha': 0.00055000000000000003}
